# Run Pretraining
The data is now in the desired format for pre-trainig of BERT. It will not be pre-trained from scratch, as the already existing German model, open-sourced by [deepset.ai](https://deepset.ai/german-bert), does not provide checkpoints. 

```bash
conda create --name=bert-pretraining tensorflow-gpu=1.14
conda activate bert-pretraining
```

## Pre-training tips and caveats - from [Google](https://github.com/google-research/bert#pre-training-tips-and-caveats)

> If using your own vocabulary, make sure to change vocab_size in bert_config.json. If you use a larger vocabulary without changing this, you will likely get NaNs when training on GPU or TPU due to unchecked out-of-bounds access.

The vocabulary size is 30,000, as in the German BERT-Base:

> If your task has a large domain-specific corpus available (e.g., "movie reviews" or "scientific papers"), it will likely be beneficial to run additional steps of pre-training on your corpus, starting from the BERT checkpoint.

> The learning rate we used in the paper was 1e-4. However, if you are doing additional steps of pre-training starting from an existing BERT checkpoint, you should use a smaller learning rate (e.g., 2e-5).

In our case, the learning rate was set to 2e-5. 

> Current BERT models are English-only, but we do plan to release a multilingual model which has been pre-trained on a lot of languages in the near future (hopefully by the end of November 2018).

> Longer sequences are disproportionately expensive because attention is quadratic to the sequence length. In other words, a batch of 64 sequences of length 512 is much more expensive than a batch of 256 sequences of length 128. The fully-connected/convolutional cost is the same, but the attention cost is far greater for the 512-length sequences. Therefore, one good recipe is to pre-train for, say, 90,000 steps with a sequence length of 128 and then for 10,000 additional steps with a sequence length of 512. The very long sequences are mostly needed to learn positional embeddings, which can be learned fairly quickly. Note that this does require generating the data twice with different values of max_seq_length.

> If you are pre-training from scratch, be prepared that pre-training is computationally expensive, especially on GPUs. If you are pre-training from scratch, our recommended recipe is to pre-train a BERT-Base on a single preemptible Cloud TPU v2, which takes about 2 weeks at a cost of about $500 USD (based on the pricing in October 2018). You will have to scale down the batch size when only training on a single Cloud TPU, compared to what was used in the paper. It is recommended to use the largest batch size that fits into TPU memory.

However, this cannot be done in our case due to dealing with sensitive patient data. 


### How many steps should be used?
Devlin et a. write: 

> We train with batch size of 256 sequences (256 sequences * 512 tokens = 128,000 tokens/batch) for 1,000,000 steps, which is approximately 40 epochs over the 3.3 billion word corpus. We use Adam with learning rate of 1e-4, β1 = 0.9, β2 = 0.999, L2 weight decay of 0.01, learning rate warm-up over the first 10,000 steps, and linear decay of the learning rate. 

Our batch size was 32 with a sequence length of 128 tokens (32 sequences * 128 tokens = 4,096 tokens/batch), whereby more was not possible with a single GPU(GTX2080ti). The resulting corpus consisted of 415,702,033 words. To achive 40 epochs, approximately 100,000 words were needed. Similar to Devlin et al, we used 1\% warmup steps.  
For pretraining, files were copied from the Google Repository

```
run_pretraining.py
modelin.py
optimization.py
```
to the `/pretraining` folder. 
To execute pre-training, the required code was wrapped in a shell script. 

```bash
#!/bin/bash

# absolute path for pretraining-folder. makes it possible to call the script form every other folder
cd ~/Documents/bert-for-radiology/pretraining

# activate the anaconda enviroment
eval "$(conda shell.bash hook)"
conda activate bert-pretraining

# give feedback
for i in {10..1}
do 
	printf $i
	printf "\n"
	sleep 0.5
done
printf "gooo.... \n\n"

# run pre-training
python run_pretraining.py \
  --input_file=../tmp/tf_examples.tfrecord-* \
  --output_dir=../tmp/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=../models/bert-base-german-cased/bert_config.json \
  --init_checkpoint=../models/bert-base-german-cased/bert_model.ckpt \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=90000 \
  --num_warmup_steps=10000 \
  --learning_rate=2e-5
```

The shell script automatically activates the required environment:



In [ ]:
!bash run_pretraining.sh

## Evaluate
Call from other enviroment. 

```bash
python run_pretraining.py \
  --input_file=../tmp/tf_examples.tfrecord-* \
  --output_dir=../tmp/pretraining_output \
  --do_train=False \
  --do_eval=True \
  --do_predict=True \
  --bert_config_file=../models/bert-base-german-cased/bert_config.json \
  --init_checkpoint=../models/bert-base-german-cased/bert_model.ckpt \
  --max_predictions_per_seq=20 
```